# Trabalho Etapa 3

## Modelo Conceitual
![](modelo_conceitual2.png)

## Modelo Lógico (adaptado para Cypher)
**No**: Sample(name, \_code_)  
**No**: Clump_Thickness(name, \_code_)  
**No**: Uniformity_of_Cell_Size(name, \_code_)  
**No**: Uniformity_of_Cell_Shape(name, \_code_)  
**No**: Marginal_Adhesion(name, \_code_)  
**No**: Single_Epithelial_Cell_Size(name, \_code_)  
**No**: Bare_Nuclei(name, \_code_)  
**No**: Bland_Chromatin(name, \_code_)  
**No**: Normal_Nucleoli(name, \_code_)  
**No**: Mitoses(name, \_code_)  
**No**: Diagnostic(name, \_code_)  

**Aresta**: Sample -> Clump_Thickness  
**Aresta**: Sample -> Uniformity_of_Cell_Size  
**Aresta**: Sample -> Uniformity_of_Cell_Shape  
**Aresta**: Sample -> Marginal_Adhesion  
**Aresta**: Sample -> Single_Epithelial_Cell_Size  
**Aresta**: Sample -> Bare_Nuclei
**Aresta**: Sample -> Bland_Chromatin  
**Aresta**: Sample -> Normal_Nucleoli  
**Aresta**: Sample -> Mitoses  
**Aresta**: Sample -> Diagnostic  
  
**Observações:** code no caso de Sample é o id da amostra. Para Diagnostic code é o valor 2 ou 4, indicando benígno e maligno respectivamente. Nos demais nós, esse atributo é um valor inteiro de 1 a10.
## Consultas Cypher


### Carregando os dados
Nós com o mesmo valor de atributo são unidos em um só
```Cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ntak1/404NotFound/master/data/breast-cancer-wisconsin.csv' AS line FIELDTERMINATOR ','
MERGE (s1:Sample {code: toInteger(line.ID_Sample), name : "Sample" })
MERGE (s2:Clump_Thickness {code:  toInteger(line.Clump_Thickness), name: "Clump_Thickness"})
MERGE (s3:Uniformity_of_Cell_Size {code:  toInteger(line.Uniformity_of_Cell_Size), name : "Uniformity_of_Cell_Size" })
MERGE(s4:Uniformity_of_Cell_Shape {code: toInteger(line.Uniformity_of_Cell_Shape) , name : "Uniformity_of_Cell_Shape"})
MERGE(s5:Marginal_Adhesion {code: toInteger(line.Marginal_Adhesion), name : "Marginal_Adhesion" })
MERGE(s6:Single_Epithelial_Cell_Size {code: toInteger(line.Single_Epithelial_Cell_Size) , name: "Single_Epithelial_Cell_Size"})
MERGE(s7:Bare_Nuclei {code: toInteger(line.Bare_Nuclei), name: "Bare_Nuclei" })
MERGE(s8:Bland_Chromatin {code: toInteger(line.Bland_Chromatin), name:"Bland_Chromatin" })
MERGE(s9:Normal_Nucleoli {code: toInteger(line.Normal_Nucleoli), name:"Normal_Nucleoli" })
MERGE(s10:Mitoses {code: toInteger(line.Mitoses), name:"Mitoses" })
MERGE(s11:Diagnostic {code: toInteger(line.Diagnostic), name: "Diagnostic" })
CREATE (s1)-[:Has]->(s2)
CREATE (s1)-[:Has]->(s3)
CREATE (s1)-[:Has]->(s4)
CREATE (s1)-[:Has]->(s5)
CREATE (s1)-[:Has]->(s6)
CREATE (s1)-[:Has]->(s7)
CREATE (s1)-[:Has]->(s8)
CREATE(s1)-[:Has]->(s9)
CREATE (s1)-[:Has]->(s10)
CREATE (s1)-[:Has]->(s11)
```

![](dados.png)

### Busca por atributos redundantes
```Cypher
match (m_a:Marginal_Adhesion)
match(m:Bare_Nuclei)
match(s1:Sample)
where (s1)-[:Has]->(m_a) and (s1)-[:Has]->(m_a) and m.code = m_a.code
return count(s1)
```
1454

### Média dos atributos das amostras malígnas
```Cypher
match (s:Sample)-[h:Has]-(n)
match (s)-[h]-(d:Diagnostic)
where d.code = 4
return avg(n.code)
```
4.0


### Média dos atributos das amostras benígnas
```Cypher
match (s:Sample)-[h:Has]-(n)
match (s)-[h]-(d:Diagnostic)
where d.code = 2
return avg(n.code)
```
2.0

### Número de amostras malígnas
```Cypher
match(s:Sample)-[:Has]->(d:Diagnostic)
where d.code = 4
return count(s)
```
239  

### Soma dos atributos das amostras benígnas
```Cypher
match (s:Sample)-[h:Has]-(n)
match (s)-[h]-(d:Diagnostic)
where d.code = 2
return sum(n.code)
```
888  


### Número de amostras malígnas
```Cypher
match(s:Sample)-[:Has]->(d:Diagnostic)
where d.code = 2
return count(s)
```
444  


### Algoritmo Closeness Centrality
"Centralidade de proximidade é uma forma de detectar nós que são capazes de espalhar informações de forma muito eficiente através de um gráfico.

A centralidade de proximidade de um nó mede sua distância média (distância inversa) a todos os outros nós. Uma alta pontuação de proximidade têm as menores distâncias para todos os outros nós." [ver referência](https://neo4j.com/docs/graph-algorithms/current/algorithms/closeness-centrality/).

```Cypher
//The Closeness Centrality algorithm
CALL algo.closeness.stream('Node', 'LINK')
YIELD nodeId, centrality

RETURN algo.getNodeById(nodeId).name AS name, algo.getNodeById(nodeId).code AS code, centrality
ORDER BY centrality DESC
LIMIT 20;
```
![](closeness.png)

### Algoritmo PageRank
Verifica quais são os nós com mais links, este algoritmo é comum em rank de páginas por sites de pesquisa. Alguns [fatos interessantes](https://pt.wikipedia.org/wiki/PageRank).
```Cypher
//Algoritmo Page Rank
CALL algo.pageRank.stream('Page', 'LINKS', {iterations:20, dampingFactor:0.85})
YIELD nodeId, score

RETURN algo.getNodeById(nodeId).name AS page,algo.getNodeById(nodeId).code AS code ,score
ORDER BY score DESC
```
![](pageRank.png)